In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import postgres_url

In [2]:
fastfood_file = "./Resources/Datafiniti_Fast_Food_Restaurants_May19.csv"
fastfood_df = pd.read_csv(fastfood_file)
fastfood_df.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.39629,-104.69699,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.08135,-108.58689,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.09148,-108.55411,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.76369,-104.77671,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.74044,-104.92636,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...


In [8]:
#select only needed columns from fastfood csv
new_fastfood_df = fastfood_df[["id","dateAdded","name","address","city","province","postalCode","categories"]].copy()
new_fastfood_df.rename(columns={
    'id': 'ffid',
    'dateAdded': 'dateadded',
    'name': 'businessname',
    'address': 'address',
    'province': 'state',
    'postalCode': 'zip',
    'categories': 'category'}, inplace=True)
new_fastfood_df.head(10)

,ffid,dateadded,businessname,address,city,state,zip,category
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,Carl's Jr.,2555 11th Avenue,Greeley,CO,80631,"Fast Food Restaurants,Hamburgers and Hot Dogs,..."
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,Del Taco,2513 Highway 6 And 50,Grand Junction,CO,81505,"Restaurant,Mexican Restaurants,Fast Food Resta..."
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,Which Wich,1125 Patterson Road,Grand Junction,CO,81506,"Sandwich Shops,Fast Food Restaurants,Restauran..."
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,Chipotle Mexican Grill,3455 N Salida Court,Aurora,CO,80011,"Fast Food Restaurants,Mexican Restaurants,Rest..."
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,Taco Bell,5225 E Colfax Avenue,Denver,CO,80220,"Fast Food Restaurants,Mexican Restaurants,Rest..."
5,AWrSaQ5RZ4Yw-wtdgcZw,2019-05-19T23:24:21Z,McDonald's,1350 W Colfax Avenue,Denver,CO,80204,"Fast Food Restaurants,Hamburgers and Hot Dogs,..."
6,AWrSaQ4EsVYjT2BJAy_I,2019-05-19T23:24:21Z,McDonald's,200 16th Street,Denver,CO,80202,"Fast Food Restaurants,Hamburgers and Hot Dogs,..."
7,AWCvDUHSIxWefVJwvq4y,2018-01-01T00:01:02Z,Taco Bell,1772 N College Ave,Fayetteville,AR,72703,"Restaurant,Carry-out food,Mexican Restaurants,..."
8,AWrSTT3sQTFama1Xpjyq,2019-05-19T22:53:58Z,Captain D's Seafood Kitchen,605 S Havana Street,Aurora,CO,80012,"Fast Food Restaurants,American Restaurants,Fam..."
9,AWkq412CIxWefVJwH_cO,2019-02-26T17:18:51Z,Shondiz,16th Glenarm Place,Denver,CO,80202,"Mexican Restaurants,Caterers,Greek Restaurants..."


In [4]:
#NOTE: believe we may need to find a dift population/zip file population ranges based on age/gender
population_data = "./Resources/population_by_zip_2010.csv"
population_data_df = pd.read_csv(population_data)
population_data_df.head()

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30.0,34.0,female,61747,8600000US61747
1,5,85.0,NaN,male,64120,8600000US64120
2,1389,30.0,34.0,male,95117,8600000US95117
3,231,60.0,61.0,female,74074,8600000US74074
4,56,0.0,4.0,female,58042,8600000US58042


In [5]:
#Create database connection
engine = create_engine(postgres_url)

In [6]:
# Confirm tables
engine.table_names()

['fastfood', 'population']

In [18]:
#Load DataFrames into database
new_fastfood_df.to_sql(name='fastfood', con=engine, if_exists='append', index=False)
population_data_df.to_sql(name='population', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "id" of relation "population" violates not-null constraint
DETAIL:  Failing row contains (null, 50, 30, 34, female, 61747, 8600000US61747).

[SQL: INSERT INTO population (population, minimum_age, maximum_age, gender, zipcode, geo_id) VALUES (%(population)s, %(minimum_age)s, %(maximum_age)s, %(gender)s, %(zipcode)s, %(geo_id)s)]
[parameters: ({'population': 50, 'minimum_age': 30.0, 'maximum_age': 34.0, 'gender': 'female', 'zipcode': 61747, 'geo_id': '8600000US61747'}, {'population': 5, 'minimum_age': 85.0, 'maximum_age': None, 'gender': 'male', 'zipcode': 64120, 'geo_id': '8600000US64120'}, {'population': 1389, 'minimum_age': 30.0, 'maximum_age': 34.0, 'gender': 'male', 'zipcode': 95117, 'geo_id': '8600000US95117'}, {'population': 231, 'minimum_age': 60.0, 'maximum_age': 61.0, 'gender': 'female', 'zipcode': 74074, 'geo_id': '8600000US74074'}, {'population': 56, 'minimum_age': 0.0, 'maximum_age': 4.0, 'gender': 'fema

In [ ]:
pd.read_sql_query('select * from fastfood', con=engine).head()

In [ ]:
pd.read_sql_query('select * from population', con=engine).head()